In [64]:
import pandas as pd
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
from future.utils import iteritems
from collections import Counter
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pymysql
from gensim.models import word2vec
from tqdm.notebook import tqdm

import sys
if sys.version_info <= (2,7):
    reload(sys)
    sys.setdefaultencoding('utf-8')
import konlpy
from konlpy.tag import Kkma, Okt, Hannanum

In [65]:
article_data = pd.read_csv('mixver10.txt', encoding='utf-8', header= None)
documents = [' '.join(i[0].split(' ')[1:]) for i in article_data.values]

as_one = ''
for document in documents:
    as_one = as_one + ' ' + document
words = as_one.split()

counts = Counter(words)

vocab = sorted(counts, key=counts.get, reverse=True)

word2idx = {word.encode("utf8").decode("utf8"): ii for ii, word in enumerate(vocab,1)}

idx2word = {ii: word for ii, word in enumerate(vocab)}

V = len(word2idx)
N = len(documents)

tf = CountVectorizer()

tf.fit_transform(documents)

tf.fit_transform(documents)[0:1].toarray()

tfidf = TfidfVectorizer(max_features = 1800, max_df=1, min_df=0)

#generate tf-idf term-document matrix
A_tfidf_sp = tfidf.fit_transform(documents)  #size D x V

tfidf_dict = tfidf.get_feature_names()

In [66]:
kkma = Kkma()
okt = Okt()
hannanum = Hannanum()
print('konlpy version = %s' % konlpy.__version__)

konlpy version = 0.5.2


In [104]:
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', password='0000',db='test', charset='utf8')
curs = conn.cursor()
# input : category, title_keyword(only noun)
# 토트백 -> 토트 백 (가방 종류 칭할 때 '백'은 띄워서 써주세요라고 명시하기)

project_category ='출판'
#title로 받은 경우
project_title = '아이와 어른이 함께 보는 출근하는 아이소파로 간 아이'

selected_word = list(set(okt.nouns(project_title)))


model = word2vec.Word2Vec.load("NaverMovie.model")
#keyword로 받은 경우
#selected_word = ['그림책', '동화']

print(selected_word)

t_cnt =0
w_cnt = 0

similar_word_list=[]
for i in selected_word:
    try:
        similar_word=model.wv.most_similar(positive=[i],topn=2)
    except:
        continue
    for j in similar_word:

        if len(j[0]) == 1:
            print()
            continue
        else:
            similar_word_list.append(j[0])
if len(selected_word) < 5:
    similar_word_list = similar_word_list + selected_word
            
similar_word_set = set(similar_word_list)    
print("similar_word_set",end='')
print(similar_word_set)
similar_word_set
res_list=[]

for i in similar_word_set:
    sql = 'select pagename,trim(title) from test.crawl where category="%s" and title like "%%%s%%" and achieve>=90;'%(project_category,i)
    curs.execute(sql)
    pagename = curs.fetchall()

    length = len(pagename)
    if length >3:
        length = 3

    for k in range(0,length):
        res_list.append((pagename[k][0], pagename[k][1]))

res_set = set(res_list)
for k in res_set:
    if k[0] == 'tumblbug':
        print("T "+k[1])
        t_cnt+=1
    elif k[0] == 'wadiz':
        print("W "+k[1])
        w_cnt+=1
    else:
        continue
    print()
print()
print("T W")
print(t_cnt, w_cnt)
if t_cnt > w_cnt:
    print("tumblbug")
elif t_cnt == w_cnt:
    print("same")
else:
    print("wadiz")
conn.close()

['어른', '아이', '소파', '간', '출근']
similar_word_set{'마련', '독립', '일력', '공기청정기', '드로잉', '감옥', '식탁', '하늘', '현상', '싱크'}
T 노정이 쓰고 드로잉메리가 그린 소설 달콤한 밤 되세요

T 인터뷰놀이X인터뷰집 가을 하늘 말갛게 뜬

T 빛나는 인생후반을 위한 추억 드로잉 “별들의 고향”

T 여성에게 국가는 없다 한남·여혐민국의 20대여성 현상

W 2차 펀딩 베스트셀러에 시들해진 그대에게_독립출판물 정기구독 서비스

T 스물아홉 결혼 말고 독립일기 당신도 혼자 살고 있나요

T 취향따라 떠나는 제주 드로잉투어컬러링북 제주

T 조선 밤하늘 별자리를 담다 조선천문도감

T 할 수 있다!독립출판제작자를 위한 대형서점 유통 가이드

T 승무원을 꿈꾸는 그대에게 아름다운 하늘을 선물합니다

T PDF 자취방 구하기 초보를 위한 내집마련 가이드북


T W
10 1
tumblbug


# keyword

In [104]:
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', password='0000',db='test', charset='utf8')
curs = conn.cursor()
# input : category, title_keyword(only noun)
# 토트백 -> 토트 백 (가방 종류 칭할 때 '백'은 띄워서 써주세요라고 명시하기)

project_category ='출판'



model = word2vec.Word2Vec.load("NaverMovie.model")
#keyword로 받은 경우
selected_word = ['그림책', '동화']

print(selected_word)

t_cnt =0
w_cnt = 0

similar_word_list=[]
for i in selected_word:
    try:
        similar_word=model.wv.most_similar(positive=[i],topn=2)
    except:
        continue
    for j in similar_word:

        if len(j[0]) == 1:
            print()
            continue
        else:
            similar_word_list.append(j[0])
if len(selected_word) < 5:
    similar_word_list = similar_word_list + selected_word
            
similar_word_set = set(similar_word_list)    
print("similar_word_set",end='')
print(similar_word_set)
similar_word_set
res_list=[]

for i in similar_word_set:
    sql = 'select pagename,trim(title) from test.crawl where category="%s" and title like "%%%s%%" and achieve>=90;'%(project_category,i)
    curs.execute(sql)
    pagename = curs.fetchall()

    length = len(pagename)
    if length >3:
        length = 3

    for k in range(0,length):
        res_list.append((pagename[k][0], pagename[k][1]))

res_set = set(res_list)
for k in res_set:
    if k[0] == 'tumblbug':
        print("T "+k[1])
        t_cnt+=1
    elif k[0] == 'wadiz':
        print("W "+k[1])
        w_cnt+=1
    else:
        continue
    print()
print()
print("T W")
print(t_cnt, w_cnt)
if t_cnt > w_cnt:
    print("tumblbug")
elif t_cnt == w_cnt:
    print("same")
else:
    print("wadiz")
conn.close()

['어른', '아이', '소파', '간', '출근']
similar_word_set{'마련', '독립', '일력', '공기청정기', '드로잉', '감옥', '식탁', '하늘', '현상', '싱크'}
T 노정이 쓰고 드로잉메리가 그린 소설 달콤한 밤 되세요

T 인터뷰놀이X인터뷰집 가을 하늘 말갛게 뜬

T 빛나는 인생후반을 위한 추억 드로잉 “별들의 고향”

T 여성에게 국가는 없다 한남·여혐민국의 20대여성 현상

W 2차 펀딩 베스트셀러에 시들해진 그대에게_독립출판물 정기구독 서비스

T 스물아홉 결혼 말고 독립일기 당신도 혼자 살고 있나요

T 취향따라 떠나는 제주 드로잉투어컬러링북 제주

T 조선 밤하늘 별자리를 담다 조선천문도감

T 할 수 있다!독립출판제작자를 위한 대형서점 유통 가이드

T 승무원을 꿈꾸는 그대에게 아름다운 하늘을 선물합니다

T PDF 자취방 구하기 초보를 위한 내집마련 가이드북


T W
10 1
tumblbug


# title

In [107]:
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', password='0000',db='test', charset='utf8')
curs = conn.cursor()
# input : category, title_keyword(only noun)
# 토트백 -> 토트 백 (가방 종류 칭할 때 '백'은 띄워서 써주세요라고 명시하기)

project_category ='출판'
#title로 받은 경우
project_title = '아이와 어른이 함께 보는 출근하는 아이소파로 간 아이'

tokenized_project_title = set(okt.nouns(project_title))
selected_word = []

for i in tokenized_project_title:
    if i in tfidf_dict:
        selected_word.append(i)
        continue

model = word2vec.Word2Vec.load("NaverMovie.model")


print(selected_word)

t_cnt =0
w_cnt = 0

similar_word_list=[]
for i in selected_word:
    try:
        similar_word=model.wv.most_similar(positive=[i],topn=2)
    except:
        continue
    for j in similar_word:

        if len(j[0]) == 1:
            print()
            continue
        else:
            similar_word_list.append(j[0])
if len(selected_word) < 5:
    similar_word_list = similar_word_list + selected_word
            
similar_word_set = set(similar_word_list)    
print("similar_word_set",end='')
print(similar_word_set)
similar_word_set
res_list=[]

for i in similar_word_set:
    sql = 'select pagename,trim(title) from test.crawl where category="%s" and title like "%%%s%%" and achieve>=90;'%(project_category,i)
    curs.execute(sql)
    pagename = curs.fetchall()

    length = len(pagename)
    if length >3:
        length = 3

    for k in range(0,length):
        res_list.append((pagename[k][0], pagename[k][1]))

res_set = set(res_list)
for k in res_set:
    if k[0] == 'tumblbug':
        print("T "+k[1])
        t_cnt+=1
    elif k[0] == 'wadiz':
        print("W "+k[1])
        w_cnt+=1
    else:
        continue
    print()
print()
print("T W")
print(t_cnt, w_cnt)
if t_cnt > w_cnt:
    print("tumblbug")
elif t_cnt == w_cnt:
    print("same")
else:
    print("wadiz")
conn.close()

['어른', '아이', '소파', '출근']
similar_word_set{'어른', '마련', '아이', '독립', '일력', '공기청정기', '드로잉', '소파', '식탁', '하늘', '출근', '싱크'}
T 연고없는 로컬로 출근한지 1년차『 논산으로 출근합니다

T 아티스트의 인사이트를 발견하다 블립 매거진아이즈원아도이

W 미국 아이비리그 출신이 만든 직장인을 위한 10분 미드 회화

T 어른들의 지적 성장 도전기 자기계발 학습지 쓸모

T 심리상담 어른이를 위한 신비한 상담소 인간관계 편

T 직관적으로 찾아보는 배색 아이디어

T 노정이 쓰고 드로잉메리가 그린 소설 달콤한 밤 되세요

T 인터뷰놀이X인터뷰집 가을 하늘 말갛게 뜬

T 빛나는 인생후반을 위한 추억 드로잉 “별들의 고향”

T 스물아홉 결혼 말고 독립일기 당신도 혼자 살고 있나요

W 2차 펀딩 베스트셀러에 시들해진 그대에게_독립출판물 정기구독 서비스

T 취향따라 떠나는 제주 드로잉투어컬러링북 제주

T 조선 밤하늘 별자리를 담다 조선천문도감

T 할 수 있다!독립출판제작자를 위한 대형서점 유통 가이드

T 승무원을 꿈꾸는 그대에게 아름다운 하늘을 선물합니다

T PDF 자취방 구하기 초보를 위한 내집마련 가이드북


T W
14 2
tumblbug
